# XGBoost Model 

This model is an XGBoost Classifier that has been trained to recognize face emotions. It makes use of the XGBoost framework to boost gradients and classify facial expressions into several emotion groups.

Model Training:
1. To Initialize,number of classes, maximum depth, learning rate, number of estimators, and number of tasks are provided.
2. To check performance on the validation set, the model is fit to the training features and labels using early stopping rounds and a verbose set.



In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import accuracy_score, classification_report

In [2]:
data = pd.read_csv('fer2013.csv')


In [3]:
# Spliting into train, val, test sets
train_data = data[data.Usage == 'Training']
test_data = data[data.Usage == 'PrivateTest']
val_data = data[data.Usage == 'PublicTest']

In [4]:
# Preprocessing images
def preprocess(data):
    X = []
    y = []
    for i in range(len(data)):
        img = data.iloc[i]['pixels'].split(' ')
        img = np.array(img, dtype='float32')
        img = img / 255.0
        X.append(img)
        y.append(data.iloc[i]['emotion'])
    X = np.array(X)
    y = np.array(y)
    return X, y

X_train, y_train = preprocess(train_data)
X_val, y_val = preprocess(val_data)
X_test, y_test = preprocess(test_data)

In [5]:
X_train = X_train.reshape(X_train.shape[0], -1)
X_val = X_val.reshape(X_val.shape[0], -1)
X_test = X_test.reshape(X_test.shape[0], -1)



# Without Regularization

Parameters:
- objective (string): The objective function to use for multi-class classification. In this case, "multi:softmax" is used.
- num_classes (int): The number of classes (emotion categories) in the classification task is given at 7.

In [6]:
clf = xgb.XGBClassifier(objective="multi:softmax", num_classes=7, max_depth=5, learning_rate=0.1, n_estimators=200, n_jobs=-1)


In [7]:
clf.fit(X_train, y_train, eval_set=[(X_val, y_val)], early_stopping_rounds=10, verbose=1)


C:\Users\nagas\anaconda3\lib\site-packages\xgboost\sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[22:22:03] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07593ffd91cd9da33-1\xgboost\xgboost-ci-windows\src\learner.cc:767: 
Parameters: { "num_classes" } are not used.

[0]	validation_0-mlogloss:1.91417
[1]	validation_0-mlogloss:1.88606
[2]	validation_0-mlogloss:1.86054
[3]	validation_0-mlogloss:1.83875
[4]	validation_0-mlogloss:1.81885
[5]	validation_0-mlogloss:1.80047
[6]	validation_0-mlogloss:1.78374
[7]	validation_0-mlogloss:1.76864
[8]	validation_0-mlogloss:1.75446
[9]	validation_0-mlogloss:1.74075
[10]	validation_0-mlogloss:1.72840
[11]	validation_0-mlogloss:1.71564
[12]	validation_0-mlogloss:1.70517
[13]	validation_0-mlogloss:1.69514
[14]	validation_0-mlogloss:1.68571
[15]	validation_0-mlogloss:1.67664
[16]	validation_0-mlogloss:1.66766
[17]	validation_0-mlogloss:1.65940
[18]	validation_0-mlogloss:1.65085
[19]	validation_0-mlogloss:1.64392
[20]	validation_0-mlogloss:1.63728
[21]	validation_0-mlogloss:1.63039
[22]	validation_0-mlogloss:1.62460
[23]

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=200, n_jobs=-1, num_classes=7,
              num_parallel_tree=None, objective='multi:softmax', ...)

In [8]:
y_pred = clf.predict(X_test)

In [13]:
y_train_pred = clf.predict(X_train)

In [14]:
accuracy = accuracy_score(y_train, y_train_pred)
print("Training Accuracy:", accuracy)

Training Accuracy: 0.8308892681737434


In [15]:
from sklearn.metrics import classification_report

target_names = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
print('Training Classification')
print(classification_report(y_train, y_train_pred, target_names=target_names, labels=[0, 1, 2, 3, 4, 5, 6]))


Training Classification
              precision    recall  f1-score   support

       Angry       0.94      0.73      0.82      3995
     Disgust       1.00      0.93      0.96       436
        Fear       0.92      0.73      0.82      4097
       Happy       0.75      0.94      0.83      7215
         Sad       0.84      0.80      0.82      4830
    Surprise       0.92      0.85      0.89      3171
     Neutral       0.78      0.84      0.81      4965

    accuracy                           0.83     28709
   macro avg       0.88      0.83      0.85     28709
weighted avg       0.84      0.83      0.83     28709



In [9]:
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy: 0.4725550292560602
              precision    recall  f1-score   support

           0       0.44      0.26      0.33       491
           1       1.00      0.27      0.43        55
           2       0.40      0.25      0.31       528
           3       0.52      0.74      0.62       879
           4       0.36      0.38      0.37       594
           5       0.64      0.60      0.62       416
           6       0.43      0.46      0.44       626

    accuracy                           0.47      3589
   macro avg       0.54      0.42      0.44      3589
weighted avg       0.47      0.47      0.46      3589



Based on this test classification, we can say that the model predicts emotions 3 (Happy), 5 (Surprise), and 6 (Neutral) reasonably well, but struggles with emotions 0 (Angry), 1 (Disgust), 2 (Fear), and 4 (Sad). More advancements can be made by modifying the model's parameters, doing feature engineering, or employing more complex methodologies. We are trying regularization in next steps.

# Regularization

In [16]:
clf = xgb.XGBClassifier(objective="multi:softmax", num_classes=7, max_depth=5, learning_rate=0.1, n_estimators=200, n_jobs=-1, reg_alpha=0.1, reg_lambda=0.1)


In [17]:
clf.fit(X_train, y_train, eval_set=[(X_val, y_val)], early_stopping_rounds=10, verbose=1)


C:\Users\nagas\anaconda3\lib\site-packages\xgboost\sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[18:18:34] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07593ffd91cd9da33-1\xgboost\xgboost-ci-windows\src\learner.cc:767: 
Parameters: { "num_classes" } are not used.

[0]	validation_0-mlogloss:1.91463
[1]	validation_0-mlogloss:1.88627
[2]	validation_0-mlogloss:1.86142
[3]	validation_0-mlogloss:1.83890
[4]	validation_0-mlogloss:1.81963
[5]	validation_0-mlogloss:1.80137
[6]	validation_0-mlogloss:1.78458
[7]	validation_0-mlogloss:1.76938
[8]	validation_0-mlogloss:1.75485
[9]	validation_0-mlogloss:1.74153
[10]	validation_0-mlogloss:1.72834
[11]	validation_0-mlogloss:1.71690
[12]	validation_0-mlogloss:1.70590
[13]	validation_0-mlogloss:1.69620
[14]	validation_0-mlogloss:1.68694
[15]	validation_0-mlogloss:1.67744
[16]	validation_0-mlogloss:1.66938
[17]	validation_0-mlogloss:1.66157
[18]	validation_0-mlogloss:1.65432
[19]	validation_0-mlogloss:1.64702
[20]	validation_0-mlogloss:1.64090
[21]	validation_0-mlogloss:1.63468
[22]	validation_0-mlogloss:1.62796
[23]

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=200, n_jobs=-1, num_classes=7,
              num_parallel_tree=None, objective='multi:softmax', ...)

In [19]:
y_pred = clf.predict(X_test)

In [20]:
y_train_pred = clf.predict(X_train)

In [21]:
from sklearn.metrics import classification_report

target_names = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
print('Training Classification')
print(classification_report(y_train, y_train_pred, target_names=target_names, labels=[0, 1, 2, 3, 4, 5, 6]))


Training Classification
              precision    recall  f1-score   support

       Angry       0.94      0.74      0.83      3995
     Disgust       1.00      0.95      0.97       436
        Fear       0.93      0.75      0.83      4097
       Happy       0.76      0.94      0.84      7215
         Sad       0.84      0.81      0.83      4830
    Surprise       0.92      0.86      0.89      3171
     Neutral       0.79      0.85      0.82      4965

    accuracy                           0.84     28709
   macro avg       0.88      0.84      0.86     28709
weighted avg       0.85      0.84      0.84     28709



In [23]:
from sklearn.metrics import classification_report

target_names = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
print('Test Classification')
print(classification_report(y_test, y_pred, target_names=target_names, labels=[0, 1, 2, 3, 4, 5, 6]))


Training Classification
              precision    recall  f1-score   support

       Angry       0.45      0.26      0.33       491
     Disgust       1.00      0.29      0.45        55
        Fear       0.41      0.26      0.32       528
       Happy       0.52      0.74      0.61       879
         Sad       0.37      0.39      0.38       594
    Surprise       0.65      0.60      0.62       416
     Neutral       0.43      0.47      0.45       626

    accuracy                           0.48      3589
   macro avg       0.55      0.43      0.45      3589
weighted avg       0.47      0.48      0.46      3589



When compared to the previous model, the regularized model performs somewhat better. There is, nevertheless, potential for improvement, particularly in reliably identifying emotions 0 (Angry), 1 (Disgust), 2 (Fear), and 4 (Sad). 

In [24]:
import joblib
joblib.dump(clf, 'XGboost_model.pkl')


['XGboost_model.pkl']